In [38]:
from ray.rllib.offline.json_reader import JsonReader
import numpy as np
import tensorflow as tf

In [31]:
input_reader = JsonReader("logs2")

2022-12-13 16:28:06,668	WARNING json_reader.py:238 -- Treating input directory as glob patterns: ['/tf/Cage/Notebooks/logs2/*.json', '/tf/Cage/Notebooks/logs2/*.zip']


In [198]:
sequence_length=5
num_episodes = 1000
windows_per_episode = 100-sequence_length

data_points = num_episodes * windows_per_episode

X = np.zeros((data_points,sequence_length,52))
Y = np.zeros((data_points,52))

index = 0
for e in range(num_episodes):
    data = input_reader.next()
    data['obs'].shape
    for i in range(0, windows_per_episode):
        ns = data['obs'][i+1]
        if i < sequence_length:
            s = np.zeros((sequence_length, 52))
            for k in range(0,i):
                s[k,:] = data['obs'][k]
        else:
            s = data['obs'][i-sequence_length:i]
            
        X[index,:,:] = s
        Y[index,:] = ns
        index += 1
index

95000

In [210]:
from keras.layers import Bidirectional
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import Bidirectional


model = Sequential()
model.add(Bidirectional(LSTM(255, activation='relu'), input_shape=(sequence_length, 52)))
model.add(Dense(52, activation='sigmoid'))
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True))

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

model.fit(X, Y, epochs=100, validation_split=0.1, verbose=1, callbacks=[callback], batch_size = 32)

/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


Train on 85500 samples, validate on 9500 samples
Epoch 1/100
85500/85500 [==============================] - 38s 443us/sample - loss: 0.0884 - val_loss: 0.0767
Epoch 2/100
40672/85500 [=============>................] - ETA: 14s - loss: 0.0763


KeyboardInterrupt



In [206]:
print(model.predict(np.zeros((1,5,52)))[0])

[4.78961803e-02 5.71352189e-07 6.26413453e-07 6.19489924e-07
 1.21430062e-01 3.10079777e-03 7.66347977e-04 4.29198053e-03
 1.94296077e-01 4.54528537e-03 6.01585535e-03 2.09340695e-02
 8.53918493e-02 8.49943142e-03 1.61375105e-03 1.93566401e-02
 1.10051809e-02 5.30886439e-07 5.41756492e-07 5.52326810e-07
 1.44799901e-02 5.97303881e-07 5.05909156e-07 6.42299710e-07
 1.71554778e-02 6.13210545e-07 5.98200018e-07 5.63436458e-07
 1.30552128e-02 4.04557306e-03 4.21508536e-04 1.29614333e-02
 1.07572496e-01 5.91283822e-07 5.25207611e-07 4.77732499e-07
 1.37692332e-01 1.74418483e-02 6.51601586e-04 2.37074103e-02
 2.85298318e-01 3.70495990e-02 2.94209621e-03 1.20227106e-01
 2.15777561e-01 3.19528542e-02 1.23869115e-03 1.04117885e-01
 2.48485148e-01 4.53932248e-02 3.95134964e-04 6.79754093e-02]


In [208]:
X[5,:,:]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 1., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 

In [209]:
print(Y[5])

[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 1.]
